In [ ]:
# create trajectories

import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

###############################################################################
# 3) Field of View Checker and Image Generation Helpers
###############################################################################

def generate_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia//2, (255, 255, 255), -1)
    return image

def generate_camera_image(camera_pose, camera_fov, point_obj, 
                          img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    return generate_image_frame(img_width, img_height, (img_x, img_y), dot_dia)

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return (abs(h_angle) <= h_fov/2) and (abs(v_angle) <= v_fov/2)

###############################################################################
# 4) Helper: Unit Vector
###############################################################################

def unit_vector(v):
    norm = np.linalg.norm(v)
    if norm < 1e-9:
        return np.array([1, 0, 0])
    return v / norm

###############################################################################
# 5) Iterative Trajectory Generation with Resampling (as before)
###############################################################################

def regenerate_candidates(i, base_positions, base_rpy, num_new_samples, reduction_factor,
                          goal_pos, lambda_decay, deg_to_rad, max_deg, max_side,
                          max_roll, max_pitch, max_yaw):
    candidates = []
    x, y, z = base_positions[i]
    roll_i, pitch_i, yaw_i = base_rpy[i]
    dx = x - goal_pos[0]
    dy = y - goal_pos[1]
    dz = z - goal_pos[2]
    r_val, az, el = cart2sph(dx, dy, dz)
    for sample in range(num_new_samples):
        frac = np.exp(-lambda_decay * (i / (len(base_positions) - 1))) if len(base_positions)>1 else 1.0
        delta_angle = frac * (max_deg * deg_to_rad) * reduction_factor
        az_var = az + random.choice([-1, 1]) * delta_angle
        el_var = el + random.choice([-1, 1]) * delta_angle
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal_pos[0] + dx_var
        y_new = goal_pos[1] + dy_var
        z_new = goal_pos[2] + dz_var

        roll_off  = frac * max_roll * reduction_factor
        pitch_off = frac * max_pitch * reduction_factor
        yaw_off   = frac * max_yaw * reduction_factor
        droll = roll_i - base_rpy[-1][0]
        dpitch = pitch_i - base_rpy[-1][1]
        dyaw = yaw_i - base_rpy[-1][2]
        droll_var = droll + random.choice([-1, 1]) * roll_off
        dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
        dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
        roll_new = base_rpy[-1][0] + droll_var
        pitch_new = base_rpy[-1][1] + dpitch_var
        yaw_new = base_rpy[-1][2] + dyaw_var

        rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
        side_len = frac * max_side
        if side_len < 1e-9:
            continue
        corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
        for j, corner in enumerate(corners):
            candidates.append({
                "trajectory_idx": i,
                "pose_6d": [corner[0], corner[1], corner[2],
                            roll_new, pitch_new, yaw_new],
                "in_fov": True  # assume resampled ones are likely in FoV
            })
    return candidates

def iterative_trajectory_generation():
    # --- A) Load main trajectory from HDF5
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                states = obs_group["joint_states"][:]  # shape: (T,7)
                positions_list.append(states[:, :3])
                orientations_list.append(states[:, 3:6])
    if not positions_list:
        print("No trajectory data found.")
        return

    positions = positions_list[0]  # (N,3)
    rpy_list = orientations_list[0]  # (N,3)
    n_points = len(positions)
    if n_points < 2:
        print("Not enough main trajectory points.")
        return

    # Precompute main trajectory's local directions.
    main_dirs = []
    for i in range(1, n_points):
        diff = positions[i] - positions[i-1]
        norm = np.linalg.norm(diff)
        if norm < 1e-9:
            main_dirs.append(np.array([1,0,0]))
        else:
            main_dirs.append(diff / norm)
    main_dirs.insert(0, main_dirs[0])

    # --- B) Base variation parameters.
    base_max_deg = 5.0
    base_max_side = 0.1
    base_max_roll = 50 * np.pi/180.0
    base_max_pitch = 10 * np.pi/180.0
    base_max_yaw = 10 * np.pi/180.0

    lambda_decay = 1.5
    camera_fov = (80, 80)
    img_width, img_height, dot_dia = 640, 480, 10
    num_samples = 10

    goal_pos = positions[-1]
    goal_rpy = rpy_list[-1]
    goal_arr = np.array(goal_pos)

    trajectories = []
    iter_idx = 0
    variation_scale = 0.1  # start with very little variation

    # Stop if candidate FoV ratio becomes too low.
    min_fov_ratio = 0.5

    while True:
        max_deg = base_max_deg * variation_scale
        max_side = base_max_side * variation_scale
        max_roll = base_max_roll * variation_scale
        max_pitch = base_max_pitch * variation_scale
        max_yaw = base_max_yaw * variation_scale

        print(f"\nIteration {iter_idx}, variation scale {variation_scale:.3f}")
        dataset = []
        total_candidates = 0
        in_fov_candidates = 0

        for i in range(n_points):
            x, y, z = positions[i]
            roll_i, pitch_i, yaw_i = rpy_list[i]
            dx = x - goal_pos[0]
            dy = y - goal_pos[1]
            dz = z - goal_pos[2]
            r_val, az, el = cart2sph(dx, dy, dz)
            droll = roll_i - goal_rpy[0]
            dpitch = pitch_i - goal_rpy[1]
            dyaw = yaw_i - goal_rpy[2]
            for sample in range(num_samples):
                frac = np.exp(-lambda_decay * (i / (n_points - 1))) if n_points>1 else 1.0
                delta_angle = frac * (max_deg * np.pi/180.0)
                az_var = az + random.choice([-1, 1]) * delta_angle
                el_var = el + random.choice([-1, 1]) * delta_angle
                dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
                x_new = goal_pos[0] + dx_var
                y_new = goal_pos[1] + dy_var
                z_new = goal_pos[2] + dz_var

                roll_off = frac * max_roll
                pitch_off = frac * max_pitch
                yaw_off = frac * max_yaw
                droll_var = droll + random.choice([-1, 1]) * roll_off
                dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
                dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
                roll_new = goal_rpy[0] + droll_var
                pitch_new = goal_rpy[1] + dpitch_var
                yaw_new = goal_rpy[2] + dyaw_var

                rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
                side_len = frac * max_side
                if side_len < 1e-9:
                    continue
                corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
                quat = rot_obj.as_quat()
                for j, corner in enumerate(corners):
                    total_candidates += 1
                    cam_pose = (corner[0], corner[1], corner[2],
                                quat[0], quat[1], quat[2], quat[3])
                    in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                    if in_fov:
                        in_fov_candidates += 1
                    dataset.append({
                        "trajectory_idx": i,
                        "pose_6d": [corner[0], corner[1], corner[2],
                                    roll_new, pitch_new, yaw_new],
                        "in_fov": in_fov,
                        "image": generate_camera_image(cam_pose, camera_fov, goal_pos,
                                                       img_width, img_height, dot_dia)
                    })
        ratio = in_fov_candidates / total_candidates if total_candidates>0 else 0
        print(f"Candidates: {total_candidates}, InFoV: {in_fov_candidates} (ratio {ratio:.2f})")
        if ratio < min_fov_ratio:
            print("FoV ratio too low. Stopping iterations.")
            break

        # Build a trajectory using a greedy chain:
        in_fov_dataset = [vp for vp in dataset if vp["in_fov"]]
        in_fov_dataset = sorted(in_fov_dataset, key=lambda vp: vp["trajectory_idx"])
        first_group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == 0]
        if not first_group:
            print("No candidates in the first group. Aborting iteration.")
            break
        current = min(first_group, key=lambda vp: np.linalg.norm(np.array(vp["pose_6d"][:3]) - positions[0]))
        chain = [current]
        current_pos = np.array(current["pose_6d"][:3])
        for i in range(1, n_points):
            group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == i]
            if not group:
                continue
            desired_dir = unit_vector(positions[i] - positions[i-1])
            best_candidate = None
            best_cost = float('inf')
            for cand in group:
                cand_pos = np.array(cand["pose_6d"][:3])
                diff = cand_pos - current_pos
                dist = np.linalg.norm(diff)
                if dist < 1e-6:
                    continue
                cand_dir = unit_vector(diff)
                angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                cost = dist + 10.0 * angle
                if cost < best_cost:
                    best_cost = cost
                    best_candidate = cand
            # If no good candidate found, try to resample for this group.
            max_attempts = 5
            attempt = 0
            while best_candidate is None and attempt < max_attempts:
                print(f"Resampling at index {i}, attempt {attempt+1}")
                new_cands = regenerate_candidates(i, positions, rpy_list, num_new_samples=5,
                                                  reduction_factor=0.5, goal_pos=goal_pos,
                                                  lambda_decay=lambda_decay, deg_to_rad=deg_to_rad,
                                                  max_deg=base_max_deg, max_side=base_max_side,
                                                  max_roll=base_max_roll, max_pitch=base_max_pitch,
                                                  max_yaw=base_max_yaw)
                candidates_by_idx[i].extend(new_cands)
                for cand in candidates_by_idx[i]:
                    cand_pos = np.array(cand["pose_6d"][:3])
                    diff = cand_pos - current_pos
                    dist = np.linalg.norm(diff)
                    if dist < 1e-6:
                        continue
                    cand_dir = unit_vector(diff)
                    angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                    cost = dist + 10.0 * angle
                    if cost < best_cost:
                        best_cost = cost
                        best_candidate = cand
                attempt += 1
            if best_candidate is None:
                print(f"Skipping index {i} due to no good candidate.")
                continue
            chain.append(best_candidate)
            current_pos = np.array(best_candidate["pose_6d"][:3])
        print(f"Iteration {iter_idx}: Generated chain with {len(chain)} points.")
        trajectories.append(chain)
        variation_scale *= 1.2
        iter_idx += 1
        if iter_idx >= 10:
            break

    # --- G) Save Videos for Each Generated Trajectory using candidate images.
    # For each trajectory chain, we use the "image" field of each candidate point
    # to create frames of a video.
    # for idx, chain in enumerate(trajectories):
    #     if not chain:
    #         continue
    #     # Use the image size from the first candidate.
    #     frame = chain[0].get("image")
    #     if frame is None:
    #         continue
    #     h, w, _ = frame.shape
    #     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    #     video_filename = f"trajectory_{idx+1}.mp4"
    #     fps = 2  # frames per second; adjust as desired
    #     out = cv2.VideoWriter(video_filename, fourcc, fps, (w, h))
    #     for cand in chain:
    #         frame = cand.get("image")
    #         if frame is None:
    #             continue
    #         # Write frame to video.
    #         out.write(frame)
    #     out.release()
    #     print(f"Saved video for trajectory {idx+1} as {video_filename}.")

    # --- H) Visualization of trajectories
    fig = go.Figure()
    main_x = [p[0] for p in positions]
    main_y = [p[1] for p in positions]
    main_z = [p[2] for p in positions]
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        line=dict(color='red', width=4),
        name='Main Trajectory'
    ))
    color_list = ["green", "blue", "orange", "purple", "brown", "cyan", "magenta", "yellow", "pink", "gray"]
    for idx, chain in enumerate(trajectories):
        chain_pts = [np.array(vp["pose_6d"][:3]) for vp in chain]
        if not chain_pts:
            continue
        chain_x = [pt[0] for pt in chain_pts]
        chain_y = [pt[1] for pt in chain_pts]
        chain_z = [pt[2] for pt in chain_pts]
        col = color_list[idx % len(color_list)]
        fig.add_trace(go.Scatter3d(
            x=chain_x, y=chain_y, z=chain_z,
            mode='lines+markers',
            line=dict(color=col, width=3),
            marker=dict(size=3),
            name=f'Trajectory {idx+1}'
        ))
    fig.update_layout(
        title="Main Trajectory (Red) and Iteratively Generated Trajectories",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

if __name__ == "__main__":
    iterative_trajectory_generation()


In [ ]:
# Create tunnel of trajectories

import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

###############################################################################
# 3) Field of View Checker and Image Generation Helpers
###############################################################################

def generate_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia//2, (255, 255, 255), -1)
    return image

def generate_camera_image(camera_pose, camera_fov, point_obj,
                          img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    return generate_image_frame(img_width, img_height, (img_x, img_y), dot_dia)

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return (abs(h_angle) <= h_fov/2) and (abs(v_angle) <= v_fov/2)

###############################################################################
# 4) Iterative Trajectory Generation (as before)
###############################################################################

def iterative_trajectory_generation():
    # Load main trajectory from HDF5
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                states = obs_group["joint_states"][:]  # shape: (T,7)
                positions_list.append(states[:, :3])
                orientations_list.append(states[:, 3:6])
    if not positions_list:
        print("No trajectory data found.")
        return None, None, None, None

    positions = positions_list[0]
    rpy_list = orientations_list[0]
    n_points = len(positions)
    if n_points < 2:
        print("Not enough main trajectory points.")
        return None, None, None, None

    # Precompute main trajectory's local directions.
    main_dirs = []
    for i in range(1, n_points):
        diff = positions[i] - positions[i-1]
        norm = np.linalg.norm(diff)
        if norm < 1e-9:
            main_dirs.append(np.array([1,0,0]))
        else:
            main_dirs.append(diff / norm)
    main_dirs.insert(0, main_dirs[0])

    # Variation base parameters.
    base_max_deg = 5.0
    base_max_side = 0.1
    base_max_roll = 50 * np.pi/180.0
    base_max_pitch = 10 * np.pi/180.0
    base_max_yaw = 10 * np.pi/180.0

    lambda_decay = 1.5
    camera_fov = (80,80)
    img_width, img_height, dot_dia = 640,480,10
    num_samples = 10

    goal_pos = positions[-1]
    goal_rpy = rpy_list[-1]
    goal_arr = np.array(goal_pos)

    trajectories = []
    iter_idx = 0
    variation_scale = 0.1  # start with very little variation

    min_fov_ratio = 0.5

    while True:
        max_deg = base_max_deg * variation_scale
        max_side = base_max_side * variation_scale
        max_roll = base_max_roll * variation_scale
        max_pitch = base_max_pitch * variation_scale
        max_yaw = base_max_yaw * variation_scale

        print(f"\nIteration {iter_idx}, variation scale {variation_scale:.3f}")
        dataset = []
        total_candidates = 0
        in_fov_candidates = 0

        for i in range(n_points):
            x, y, z = positions[i]
            roll_i, pitch_i, yaw_i = rpy_list[i]
            dx = x - goal_pos[0]
            dy = y - goal_pos[1]
            dz = z - goal_pos[2]
            r_val, az, el = cart2sph(dx, dy, dz)
            droll = roll_i - goal_rpy[0]
            dpitch = pitch_i - goal_rpy[1]
            dyaw = yaw_i - goal_rpy[2]
            for sample in range(num_samples):
                frac = np.exp(-lambda_decay * (i / (n_points - 1))) if n_points>1 else 1.0
                delta_angle = frac * (max_deg * np.pi/180.0)
                az_var = az + random.choice([-1, 1]) * delta_angle
                el_var = el + random.choice([-1, 1]) * delta_angle
                dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
                x_new = goal_pos[0] + dx_var
                y_new = goal_pos[1] + dy_var
                z_new = goal_pos[2] + dz_var

                roll_off = frac * max_roll
                pitch_off = frac * max_pitch
                yaw_off = frac * max_yaw
                droll_var = droll + random.choice([-1, 1]) * roll_off
                dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
                dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
                roll_new = goal_rpy[0] + droll_var
                pitch_new = goal_rpy[1] + dpitch_var
                yaw_new = goal_rpy[2] + dyaw_var

                rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
                side_len = frac * max_side
                if side_len < 1e-9:
                    continue
                corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
                quat = rot_obj.as_quat()
                for j, corner in enumerate(corners):
                    total_candidates += 1
                    cam_pose = (corner[0], corner[1], corner[2],
                                quat[0], quat[1], quat[2], quat[3])
                    in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                    if in_fov:
                        in_fov_candidates += 1
                    dataset.append({
                        "trajectory_idx": i,
                        "pose_6d": [corner[0], corner[1], corner[2],
                                    roll_new, pitch_new, yaw_new],
                        "in_fov": in_fov,
                        "image": generate_camera_image(cam_pose, camera_fov, goal_pos,
                                                       img_width, img_height, dot_dia)
                    })
        ratio = in_fov_candidates / total_candidates if total_candidates>0 else 0
        print(f"Candidates: {total_candidates}, InFoV: {in_fov_candidates} (ratio {ratio:.2f})")
        if ratio < min_fov_ratio:
            print("FoV ratio too low. Stopping iterations.")
            break

        # Build a trajectory using a greedy chain:
        in_fov_dataset = [vp for vp in dataset if vp["in_fov"]]
        in_fov_dataset = sorted(in_fov_dataset, key=lambda vp: vp["trajectory_idx"])
        first_group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == 0]
        if not first_group:
            print("No candidates in the first group. Aborting iteration.")
            break
        current = min(first_group, key=lambda vp: np.linalg.norm(np.array(vp["pose_6d"][:3]) - positions[0]))
        chain = [current]
        current_pos = np.array(current["pose_6d"][:3])
        for i in range(1, n_points):
            group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == i]
            if not group:
                continue
            desired_dir = unit_vector(positions[i] - positions[i-1])
            best_candidate = None
            best_cost = float('inf')
            for cand in group:
                cand_pos = np.array(cand["pose_6d"][:3])
                diff = cand_pos - current_pos
                dist = np.linalg.norm(diff)
                if dist < 1e-6:
                    continue
                cand_dir = unit_vector(diff)
                angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                cost = dist + 10.0 * angle
                if cost < best_cost:
                    best_cost = cost
                    best_candidate = cand
            max_attempts = 5
            attempt = 0
            while best_candidate is None and attempt < max_attempts:
                print(f"Resampling at index {i}, attempt {attempt+1}")
                new_cands = regenerate_candidates(i, positions, rpy_list, num_new_samples=5,
                                                  reduction_factor=0.5, goal_pos=goal_pos,
                                                  lambda_decay=lambda_decay, deg_to_rad=deg_to_rad,
                                                  max_deg=base_max_deg, max_side=base_max_side,
                                                  max_roll=base_max_roll, max_pitch=base_max_pitch,
                                                  max_yaw=base_max_yaw)
                # Append these new candidates to the group.
                candidates_by_idx[i].extend(new_cands)
                for cand in candidates_by_idx[i]:
                    cand_pos = np.array(cand["pose_6d"][:3])
                    diff = cand_pos - current_pos
                    dist = np.linalg.norm(diff)
                    if dist < 1e-6:
                        continue
                    cand_dir = unit_vector(diff)
                    angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                    cost = dist + 10.0 * angle
                    if cost < best_cost:
                        best_cost = cost
                        best_candidate = cand
                attempt += 1
            if best_candidate is None:
                print(f"Skipping index {i} due to no good candidate.")
                continue
            chain.append(best_candidate)
            current_pos = np.array(best_candidate["pose_6d"][:3])
        print(f"Iteration {iter_idx}: Generated chain with {len(chain)} points.")
        trajectories.append(chain)
        variation_scale *= 1.2
        iter_idx += 1
        if iter_idx >= 10:
            break

    # --- F) Save Videos for Each Generated Trajectory using camera images.
    # for idx, chain in enumerate(trajectories):
    #     if not chain:
    #         continue
    #     frame = chain[0].get("image")
    #     if frame is None:
    #         continue
    #     h, w, _ = frame.shape
    #     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    #     video_filename = f"trajectory_{idx+1}.mp4"
    #     fps = 2
    #     out = cv2.VideoWriter(video_filename, fourcc, fps, (w, h))
    #     for cand in chain:
    #         frame = cand.get("image")
    #         if frame is None:
    #             continue
    #         out.write(frame)
    #     out.release()
    #     print(f"Saved video for trajectory {idx+1} as {video_filename}.")

    # --- G) Visualization of trajectories.
    fig = go.Figure()
    main_x = [p[0] for p in positions]
    main_y = [p[1] for p in positions]
    main_z = [p[2] for p in positions]
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        line=dict(color='red', width=4),
        name='Main Trajectory'
    ))
    color_list = ["green", "blue", "orange", "purple", "brown", "cyan", "magenta", "yellow", "pink", "gray"]
    for idx, chain in enumerate(trajectories):
        chain_pts = [np.array(vp["pose_6d"][:3]) for vp in chain]
        if not chain_pts:
            continue
        chain_x = [pt[0] for pt in chain_pts]
        chain_y = [pt[1] for pt in chain_pts]
        chain_z = [pt[2] for pt in chain_pts]
        col = color_list[idx % len(color_list)]
        fig.add_trace(go.Scatter3d(
            x=chain_x, y=chain_y, z=chain_z,
            mode='lines+markers',
            line=dict(color=col, width=3),
            marker=dict(size=3),
            name=f'Trajectory {idx+1}'
        ))
    fig.update_layout(
        title="Main Trajectory (Red) and Iteratively Generated Trajectories",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

    return trajectories, positions, main_dirs, dataset

###############################################################################
# 7) Generate 3D Boundary (Tunnel) Enclosing All Trajectories
###############################################################################
def generate_boundary(trajectories, positions, main_dirs, num_circle_pts=20):
    """
    Generate a 3D boundary (tunnel) that encloses all generated trajectories.
    For each main trajectory index i, we:
      - Collect all candidate points from any trajectory with trajectory_idx==i.
      - Compute the maximum Euclidean distance from the main trajectory point.
      - Generate a circle (in the plane perpendicular to the main direction) with that radius.
    Returns a list of circle curves (each as (x,y,z) arrays).
    """
    n_points = len(positions)
    boundary_circles = []
    for i in range(n_points):
        # Collect all candidate points from all trajectories with trajectory_idx == i.
        pts = []
        for traj in trajectories:
            for cand in traj:
                if cand["trajectory_idx"] == i:
                    pts.append(np.array(cand["pose_6d"][:3]))
        if not pts:
            # If no candidate points for this index, use the main trajectory point.
            center = np.array(positions[i])
            radius = 0.0
        else:
            pts = np.array(pts)
            center = np.array(positions[i])
            dists = np.linalg.norm(pts - center, axis=1)
            radius = dists.max()
        # Get main direction at index i.
        if i < len(main_dirs):
            normal = main_dirs[i]
        else:
            normal = main_dirs[-1]
        # Generate two orthonormal vectors u,v perpendicular to normal.
        # Choose an arbitrary vector that is not parallel to normal.
        arbitrary = np.array([1, 0, 0]) if abs(normal[0]) < 0.9 else np.array([0, 1, 0])
        u = np.cross(normal, arbitrary)
        u = unit_vector(u)
        v = np.cross(normal, u)
        v = unit_vector(v)
        # Generate circle points.
        theta = np.linspace(0, 2*np.pi, num_circle_pts)
        circle_pts = np.array([center + radius * (np.cos(t)*u + np.sin(t)*v) for t in theta])
        boundary_circles.append(circle_pts)
    return boundary_circles

###############################################################################
# 8) Visualization of Boundary
###############################################################################
def visualize_boundary(boundary_circles, positions):
    # Plot the main trajectory and the boundary circles.
    main_x = [p[0] for p in positions]
    main_y = [p[1] for p in positions]
    main_z = [p[2] for p in positions]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        line=dict(color='red', width=4),
        name='Main Trajectory'
    ))
    # Plot each circle as a closed loop.
    for circle in boundary_circles:
        fig.add_trace(go.Scatter3d(
            x=circle[:,0], y=circle[:,1], z=circle[:,2],
            mode='lines',
            line=dict(color='blue', width=2),
            name='Boundary'
        ))
    fig.update_layout(
        title="3D Boundary Enclosing Generated Trajectories",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

###############################################################################
# Main Execution
###############################################################################
if __name__ == "__main__":
    trajectories, positions, main_dirs, dataset = iterative_trajectory_generation()
    if trajectories is None:
        exit()
    # Generate the 3D boundary from the generated trajectories.
    boundary_circles = generate_boundary(trajectories, positions, main_dirs, num_circle_pts=30)
    visualize_boundary(boundary_circles, positions)


In [ ]:
# Create convex hull (Safe set)

import random
import h5py
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull
from scipy.spatial.transform import Rotation as R
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict

###############################################################################
# 1) Cartesian <-> Spherical Conversions
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

def unit_vector(vec):
        normv = np.linalg.norm(vec)
        if normv < 1e-9:
            return np.array([1,0,0])  # fallback
        return vec / normv


###############################################################################
# 2) Cube Creation (Generate Cube Corners as Candidate Viewpoints)
###############################################################################

def make_cube(cx, cy, cz, rotation, side=0.005):
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

###############################################################################
# 3) Field of View Checker and Image Generation Helpers
###############################################################################

def generate_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia//2, (255,255,255), -1)
    return image

def generate_camera_image(camera_pose, camera_fov, point_obj, 
                          img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    return generate_image_frame(img_width, img_height, (img_x, img_y), dot_dia)

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj)-cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj)-cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return (abs(h_angle) <= h_fov/2) and (abs(v_angle) <= v_fov/2)

###############################################################################
# 4) Iterative Trajectory Generation (Candidate-based)
###############################################################################

def iterative_trajectory_generation():
    # Load main trajectory from HDF5 file.
    file_path = "demo_duck_feb12.hdf5"
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        for demo_name in demo_keys[:1]:
            demo_data = f["data"][demo_name]
            obs_group = demo_data["obs"]
            if "joint_states" in obs_group:
                states = obs_group["joint_states"][:]  # shape: (T,7)
                positions_list.append(states[:, :3])
                orientations_list.append(states[:, 3:6])
    if not positions_list:
        print("No trajectory data found.")
        return None, None, None, None

    positions = positions_list[0]
    rpy_list = orientations_list[0]
    n_points = len(positions)
    if n_points < 2:
        print("Not enough main trajectory points.")
        return None, None, None, None

    # Precompute main trajectory's local directions.
    main_dirs = []
    for i in range(1, n_points):
        diff = positions[i] - positions[i-1]
        norm = np.linalg.norm(diff)
        if norm < 1e-9:
            main_dirs.append(np.array([1,0,0]))
        else:
            main_dirs.append(diff / norm)
    main_dirs.insert(0, main_dirs[0])

    # Base variation parameters.
    base_max_deg = 5.0
    base_max_side = 0.1
    base_max_roll = 50 * np.pi/180.0
    base_max_pitch = 10 * np.pi/180.0
    base_max_yaw = 10 * np.pi/180.0

    lambda_decay = 1.5
    camera_fov = (80,80)
    img_width, img_height, dot_dia = 640,480,10
    num_samples = 10

    goal_pos = positions[-1]
    goal_rpy = rpy_list[-1]
    goal_arr = np.array(goal_pos)

    trajectories = []
    iter_idx = 0
    variation_scale = 0.1
    min_fov_ratio = 0.5

    while True:
        max_deg = base_max_deg * variation_scale
        max_side = base_max_side * variation_scale
        max_roll = base_max_roll * variation_scale
        max_pitch = base_max_pitch * variation_scale
        max_yaw = base_max_yaw * variation_scale

        print(f"\nIteration {iter_idx}, variation scale {variation_scale:.3f}")
        dataset = []
        total_candidates = 0
        in_fov_candidates = 0

        for i in range(n_points):
            x, y, z = positions[i]
            roll_i, pitch_i, yaw_i = rpy_list[i]
            dx = x - goal_pos[0]
            dy = y - goal_pos[1]
            dz = z - goal_pos[2]
            r_val, az, el = cart2sph(dx, dy, dz)
            droll = roll_i - goal_rpy[0]
            dpitch = pitch_i - goal_rpy[1]
            dyaw = yaw_i - goal_rpy[2]
            for sample in range(num_samples):
                frac = np.exp(-lambda_decay * (i / (n_points - 1))) if n_points>1 else 1.0
                delta_angle = frac * (max_deg * np.pi/180.0)
                az_var = az + random.choice([-1, 1]) * delta_angle
                el_var = el + random.choice([-1, 1]) * delta_angle
                dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
                x_new = goal_pos[0] + dx_var
                y_new = goal_pos[1] + dy_var
                z_new = goal_pos[2] + dz_var

                roll_off = frac * max_roll
                pitch_off = frac * max_pitch
                yaw_off = frac * max_yaw
                droll_var = droll + random.choice([-1, 1]) * roll_off
                dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
                dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
                roll_new = goal_rpy[0] + droll_var
                pitch_new = goal_rpy[1] + dpitch_var
                yaw_new = goal_rpy[2] + dyaw_var

                rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
                side_len = frac * max_side
                if side_len < 1e-9:
                    continue
                corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
                quat = rot_obj.as_quat()
                for j, corner in enumerate(corners):
                    total_candidates += 1
                    cam_pose = (corner[0], corner[1], corner[2],
                                quat[0], quat[1], quat[2], quat[3])
                    in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                    if in_fov:
                        in_fov_candidates += 1
                    dataset.append({
                        "trajectory_idx": i,
                        "pose_6d": [corner[0], corner[1], corner[2],
                                    roll_new, pitch_new, yaw_new],
                        "in_fov": in_fov,
                        "image": generate_camera_image(cam_pose, camera_fov, goal_pos,
                                                       img_width, img_height, dot_dia)
                    })
        ratio = in_fov_candidates / total_candidates if total_candidates>0 else 0
        print(f"Candidates: {total_candidates}, InFoV: {in_fov_candidates} (ratio {ratio:.2f})")
        if ratio < min_fov_ratio:
            print("FoV ratio too low. Stopping iterations.")
            break

        # Build a trajectory using a greedy chain.
        in_fov_dataset = [vp for vp in dataset if vp["in_fov"]]
        in_fov_dataset = sorted(in_fov_dataset, key=lambda vp: vp["trajectory_idx"])
        first_group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == 0]
        if not first_group:
            print("No candidates in the first group. Aborting iteration.")
            break
        current = min(first_group, key=lambda vp: np.linalg.norm(np.array(vp["pose_6d"][:3]) - positions[0]))
        chain = [current]
        current_pos = np.array(current["pose_6d"][:3])
        for i in range(1, n_points):
            group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == i]
            if not group:
                continue
            desired_dir = unit_vector(positions[i] - positions[i-1])
            best_candidate = None
            best_cost = float('inf')
            for cand in group:
                cand_pos = np.array(cand["pose_6d"][:3])
                diff = cand_pos - current_pos
                dist = np.linalg.norm(diff)
                if dist < 1e-6:
                    continue
                cand_dir = unit_vector(diff)
                angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                cost = dist + 10.0 * angle
                if cost < best_cost:
                    best_cost = cost
                    best_candidate = cand
            max_attempts = 5
            attempt = 0
            while best_candidate is None and attempt < max_attempts:
                print(f"Resampling at index {i}, attempt {attempt+1}")
                new_cands = regenerate_candidates(i, positions, rpy_list, num_new_samples=5,
                                                  reduction_factor=0.5, goal_pos=goal_pos,
                                                  lambda_decay=lambda_decay, deg_to_rad=deg_to_rad,
                                                  max_deg=base_max_deg, max_side=base_max_side,
                                                  max_roll=base_max_roll, max_pitch=base_max_pitch,
                                                  max_yaw=base_max_yaw)
                for nc in new_cands:
                    dataset.append(nc)
                    if nc["in_fov"]:
                        in_fov_dataset.append(nc)
                for cand in [vp for vp in in_fov_dataset if vp["trajectory_idx"]==i]:
                    cand_pos = np.array(cand["pose_6d"][:3])
                    diff = cand_pos - current_pos
                    dist = np.linalg.norm(diff)
                    if dist < 1e-6:
                        continue
                    cand_dir = unit_vector(diff)
                    angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                    cost = dist + 10.0 * angle
                    if cost < best_cost:
                        best_cost = cost
                        best_candidate = cand
                attempt += 1
            if best_candidate is None:
                print(f"Skipping index {i} due to no good candidate.")
                continue
            chain.append(best_candidate)
            current_pos = np.array(best_candidate["pose_6d"][:3])
        print(f"Iteration {iter_idx}: Generated chain with {len(chain)} points.")
        trajectories.append(chain)
        variation_scale *= 1.2
        iter_idx += 1
        if iter_idx >= 10:
            break

    print(f"Generated {len(trajectories)} trajectories.")
    return trajectories, positions, main_dirs, dataset

###############################################################################
# 7) Generate 3D Surface Boundary from Merged Trajectory Points
###############################################################################
def generate_surface_boundary(trajectories):
    all_points = []
    for traj in trajectories:
        for cand in traj:
            pt = np.array(cand["pose_6d"][:3])
            all_points.append(pt)
    all_points = np.array(all_points)
    hull = ConvexHull(all_points)
    return all_points, hull

###############################################################################
# 8) Visualization of Boundary and Main Trajectory
###############################################################################
def visualize_boundary_and_main(all_points, hull, positions):
    main_x = [p[0] for p in positions]
    main_y = [p[1] for p in positions]
    main_z = [p[2] for p in positions]
    
    fig = go.Figure()
    # Plot the convex hull as a mesh.
    fig.add_trace(go.Mesh3d(
        x=all_points[:,0],
        y=all_points[:,1],
        z=all_points[:,2],
        i=hull.simplices[:,0],
        j=hull.simplices[:,1],
        k=hull.simplices[:,2],
        opacity=0.3,
        color='lightblue',
        name='Boundary'
    ))
    # Plot the main trajectory as a thin line.
    fig.add_trace(go.Scatter3d(
        x=main_x, y=main_y, z=main_z,
        mode='lines+markers',
        line=dict(color='red', width=2),  # small width line
        marker=dict(size=2),
        name='Main Trajectory'
    ))
    fig.update_layout(
        title="Main Trajectory Within 3D Boundary",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

###############################################################################
# 9) Main Execution
###############################################################################
if __name__ == "__main__":
    trajectories, positions, main_dirs, dataset = iterative_trajectory_generation()
    if trajectories is None:
        exit()
    all_points, hull = generate_surface_boundary(trajectories)
    visualize_boundary_and_main(all_points, hull, positions)
